<a href="https://colab.research.google.com/github/mikeonly/nma_easy_cabbage/blob/master/process_isi_metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/My Drive/Colab Notebooks/NMA/Data

In [2]:
import os, requests

fname = []
fname = ['steinmetz_st.npz']

url = ["https://osf.io/4bjns/download"]

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)

In [3]:
import numpy as np
import pickle

dat_ST = np.load('steinmetz_st.npz', allow_pickle=True)['dat']

In [10]:
isi = [] #contains lists of isi
metadata = [] #contains corresponding metadata in tuple (session, trial, neuron)
# iterate over sessions
for s_no in range(0,dat_ST.shape[0]):
  #iterate over sessions for a given trial
  for t_no in range(0, dat_ST[s_no]['ss'].shape[1]): 
    #iterate over neurons for a given trial in a session
    for n_no in range(0, dat_ST[s_no]['ss'].shape[0]):
      #get spike times for the selected neuron
      spikes = dat_ST[s_no]['ss'][n_no][t_no]
      #check if atleast two the neuron spikes atleast twice
      if (len(spikes) > 1):
        n_isi = np.diff(spikes)
        #do NOT use numpy here as jagged arrays are not allowed
        isi.append(list(n_isi))
        metadata.append((s_no, t_no, n_no))

In [ ]:
#use the following snippet for saving the two arrays
#Note: isi is NOT a numpy array, hence pickle is preferred

with open("isi.pkl", "wb") as f:
  pickle.dump(isi, f)

with open("isi_metadata.pkl", "wb") as f:
  pickle.dump(metadata, f)